# 🎯 Treinamento YOLOv8 - Detecção de Adesivo
## Google Colab - GPU Acelerado

**Importante:** Antes de começar, vá em `Runtime > Change runtime type` e selecione **GPU** como acelerador.

## 1️⃣ Instalação de Dependências

In [ ]:
# Instalar ultralytics (YOLOv8)
!pip install ultralytics -q

## 2️⃣ Verificar GPU

In [ ]:
import torch
from ultralytics import YOLO

print("="*60)
print("VERIFICAÇÃO DE AMBIENTE")
print("="*60)

print(f"\n✓ PyTorch versão: {torch.__version__}")
print(f"✓ CUDA disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"✓ Memória GPU Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    device = 0
else:
    print("⚠ GPU não detectada. Verifique se selecionou GPU em Runtime > Change runtime type")
    device = 'cpu'

## 3️⃣ Upload do Dataset

**Opção A:** Faça upload manual do dataset compactado (ZIP)

In [ ]:
from google.colab import files
import zipfile
import os

print("📦 Faça upload do arquivo ZIP contendo o dataset...")
uploaded = files.upload()

# Descompactar
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📂 Descompactando {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✓ Dataset descompactado!")
        os.remove(filename)  # Remove o ZIP após descompactar

**Opção B:** Montar Google Drive (se o dataset estiver lá)

In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Copiar dataset do Drive (ajuste o caminho conforme necessário)
# !cp -r /content/drive/MyDrive/seu_dataset ./dataset

## 4️⃣ Criar arquivo data.yaml

In [ ]:
# Criar arquivo de configuração do dataset
data_yaml = """path: ./dataset
train: images/train
val: images/val

nc: 2
names: ['com_adesivo', 'sem_adesivo']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

print("✓ Arquivo data.yaml criado!")

## 5️⃣ Verificar estrutura do dataset

In [ ]:
import os

# Verificar se as pastas existem
print("📁 Verificando estrutura do dataset...\n")

paths_to_check = [
    'dataset/images/train',
    'dataset/images/val',
    'dataset/labels/train',
    'dataset/labels/val'
]

for path in paths_to_check:
    if os.path.exists(path):
        count = len(os.listdir(path))
        print(f"✓ {path}: {count} arquivos")
    else:
        print(f"⚠ {path}: NÃO ENCONTRADO")

## 6️⃣ TREINAMENTO 🚀

In [ ]:
print("="*60)
print("TREINAMENTO - DETECÇÃO DE ADESIVO")
print("="*60)
print("\nIniciando treinamento...")
print("="*60)

# Carregar modelo pré-treinado YOLOv8s
model = YOLO('yolov8s.pt')

# Treinar modelo
results = model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=4,            # Ajuste conforme a GPU do Colab
    patience=20,
    device=device,
    project='adesivo_detection',
    name='run1',
    save=True,
    plots=True,         # Ativado para visualizar gráficos
    workers=2,          # Colab suporta mais workers
    verbose=True
)

print("\n" + "="*60)
print("✓ TREINAMENTO CONCLUÍDO!")
print("="*60)
print(f"\n📊 Resultados salvos em: adesivo_detection/run1/")
print(f"🏆 Melhor modelo: adesivo_detection/run1/weights/best.pt")
print(f"📈 Gráficos: adesivo_detection/run1/*.png")

## 7️⃣ Validação do Modelo

In [ ]:
print("="*60)
print("Validando modelo...")
print("="*60)

# Carregar melhor modelo treinado
model = YOLO('adesivo_detection/run1/weights/best.pt')
metrics = model.val()

print(f"\n📊 MÉTRICAS FINAIS:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precisão: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

print("\n✓ Validação concluída!")

## 8️⃣ Visualizar Gráficos de Treinamento

In [ ]:
from IPython.display import Image, display
import os

# Mostrar gráficos de resultados
results_path = 'adesivo_detection/run1'

print("📊 GRÁFICOS DE TREINAMENTO:\n")

# Listar todos os gráficos disponíveis
plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'P_curve.png', 'R_curve.png', 'PR_curve.png']

for plot in plots:
    plot_path = os.path.join(results_path, plot)
    if os.path.exists(plot_path):
        print(f"\n{plot}:")
        display(Image(filename=plot_path))

## 9️⃣ Download do Modelo Treinado

In [ ]:
from google.colab import files
import shutil

# Criar ZIP com os modelos e resultados
print("📦 Compactando resultados...")
shutil.make_archive('adesivo_detection_results', 'zip', 'adesivo_detection/run1')

print("⬇️ Fazendo download...")
files.download('adesivo_detection_results.zip')

print("\n✓ Download concluído!")
print("\nO arquivo ZIP contém:")
print("  - weights/best.pt (melhor modelo)")
print("  - weights/last.pt (último checkpoint)")
print("  - Todos os gráficos e métricas")

## 🔟 (Opcional) Salvar no Google Drive

In [ ]:
# Copiar resultados para o Google Drive
# !cp -r adesivo_detection/run1 /content/drive/MyDrive/adesivo_detection_results

print("✓ Resultados salvos no Google Drive!")

---

## 📝 Notas Importantes:

1. **GPU:** Certifique-se de que a GPU está ativada em `Runtime > Change runtime type`
2. **Tempo de execução:** O Colab gratuito tem limite de tempo de sessão (~12h)
3. **Ajuste de batch:** Se der erro de memória, reduza o `batch` para 2 ou 1
4. **Dataset:** A estrutura deve ser:
   ```
   dataset/
   ├── images/
   │   ├── train/
   │   └── val/
   └── labels/
       ├── train/
       └── val/
   ```

## 🚀 Próximos Passos:

1. Faça download do modelo treinado (`best.pt`)
2. Use o modelo no seu ambiente local com `test.py`
3. Se necessário, faça fine-tuning com mais épocas ou dados